<h3>Import kruft

In [1]:
!python -m site


sys.path = [
    '/home/mohcine/Projetcs/pysparkKafkaNifi/Spark_from_Spark/PyGotham_Spark_Streaming_demo/notebooks',
    '/home/mohcine/Software/spark-2.3.1-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip',
    '/home/mohcine/Software/spark-2.3.1-bin-hadoop2.7/python',
    '/home/mohcine/Software/anaconda2/lib/python27.zip',
    '/home/mohcine/Software/anaconda2/lib/python2.7',
    '/home/mohcine/Software/anaconda2/lib/python2.7/plat-linux2',
    '/home/mohcine/Software/anaconda2/lib/python2.7/lib-tk',
    '/home/mohcine/Software/anaconda2/lib/python2.7/lib-old',
    '/home/mohcine/Software/anaconda2/lib/python2.7/lib-dynload',
    '/home/mohcine/Software/anaconda2/lib/python2.7/site-packages',
]
USER_BASE: '/home/mohcine/.local' (exists)
USER_SITE: '/home/mohcine/.local/lib/python2.7/site-packages' (doesn't exist)
ENABLE_USER_SITE: True


In [2]:
import pyspark as ps
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc, asc
from pyspark.streaming import StreamingContext

from pyspark.ml.feature import StopWordsRemover

#plotting
import matplotlib.pyplot as plt
import seaborn as sn
%matplotlib inline

import pandas as pd
import socket

<h3>Initialize Spark Context

In [3]:
conf = SparkConf().setMaster("local[2]").setAppName("hello_pygotham")
sc = SparkContext(conf=conf)
sc

<SparkContext master=local[2] appName=hello_pygotham>

In [4]:
#SQLContext allows us to query results with SQL-like syntax
sqlContext = SQLContext(sc)

<h3>Initialize Spark Streaming Context

In [5]:
ssc = StreamingContext(sc, 5)      #5 second microbatches
ssc

<h3>Import external dataset

We'll be using the ANEW (Affective Norms of English Words) data set. More info: http://www.uvm.edu/pdodds/teaching/courses/2009-08UVM-300/docs/others/everything/bradley1999a.pdf.

**Measurements of mean reactions to words (1-9 scale)**
- Good to bad (psychological valence)
- Active to passive (arousal)
- Strong to weak (dominance)

In [6]:
#load ANEW data set
anew_dict = {}
with open("all.csv", "r") as rf:
    anew = [x for x in rf.read().splitlines()]
    for line in anew:
        (key, value) = line.split(",", 1)
        value = value.split(",")
        anew_dict[key] = value

In [7]:
#contains mean valence, dominance and arousal scores for a corpus of English words
anew_dict.get("Description")

['Word No.',
 'Valence Mean',
 'Valence SD',
 'Arousal Mean',
 'Arousal SD',
 'Dominance Mean',
 'Dominance SD',
 'Word Frequency']

In [8]:
anew_dict.get("grin")

['773', '7.40', '1.87', '5.27', '2.64', '6.00', '1.86', '13']

In [9]:
anew_dict.get("horse")

['214', '5.89', '1.55', '3.89', '2.17', '4.67', '1.60', '117']

<h3>Define streaming data input

In [16]:
#see bin/TweetRead.py. ifconfig in terminal will get you host ip
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.connect(("8.8.8.8", 80))
ip = (sock.getsockname()[0])
port = 5555

#our data input
stream = ssc.socketTextStream(ip, port)

Py4JJavaError: An error occurred while calling o29.socketTextStream.
: java.lang.IllegalStateException: Adding new inputs, transformations, and output operations after starting a context is not supported
	at org.apache.spark.streaming.dstream.DStream.validateAtInit(DStream.scala:224)
	at org.apache.spark.streaming.dstream.DStream.<init>(DStream.scala:66)
	at org.apache.spark.streaming.dstream.InputDStream.<init>(InputDStream.scala:45)
	at org.apache.spark.streaming.dstream.ReceiverInputDStream.<init>(ReceiverInputDStream.scala:42)
	at org.apache.spark.streaming.dstream.SocketInputDStream.<init>(SocketInputDStream.scala:40)
	at org.apache.spark.streaming.StreamingContext.socketStream(StreamingContext.scala:322)
	at org.apache.spark.streaming.StreamingContext$$anonfun$socketTextStream$1.apply(StreamingContext.scala:303)
	at org.apache.spark.streaming.StreamingContext$$anonfun$socketTextStream$1.apply(StreamingContext.scala:303)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOpe

In [17]:
ip


'192.168.0.94'

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/mohcine/Software/anaconda2/lib/python2.7/site-packages/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/home/mohcine/Software/anaconda2/lib/python2.7/site-packages/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:35539)
Traceback (most recent call last):
  File "/home/mohcine/Software/anaconda2/lib/python2.7/site-packages/py4j/java_gateway.py", line 1071, in start
    self._authenticate_connection()
  File "/home/mohcine/Software/anaconda2/lib/python2.7/site-packages/py4j/java_gateway.py", line 1087, in _authenticate_connection
    answer = self.send_command(cmd)
  File "/home/mohcine/Software/anaconda2/lib/python2.7/site-packages/py4j/java_gateway.

In [11]:
window_time = 10
lines = stream.window( window_time )

In [12]:
#not a comprehensive set, but we're just fooling around anyway
stopwords = StopWordsRemover.loadDefaultStopWords("english")

In [13]:
#attempt to use ANEW dataset
(lines.flatMap(lambda x : x.split(" "))   #split each tweet by space
              .filter(lambda x: x.lower() not in stopwords)  #remove english stopwords, words not in ANEW
              .filter(lambda x : x.lower() in anew_dict)
              .map(lambda word : (word.lower(), anew_dict.get(word.lower())[3], anew_dict.get(word.lower())[5], 1))   #lookup anew arousal and dominance scores, if available
              .foreachRDD(lambda rdd : rdd.toDF().registerTempTable("tweets")))

<b>Note: Nothing's happening just yet.

In [14]:
ssc.start()

<b>Now stuff is happening! Yay!

<h3>Some simple visualizations



In [15]:
#plot scatterplots of arousal and dominance
import datetime
from IPython import display
import time

count = 0
while count < 10:
    try:
        time.sleep(window_time)
        #display.clear_output(wait=True)
        print(datetime.datetime.now())
        myrows = sqlContext.sql("select * from tweets where _2 is not null")
        myrows = myrows.groupby(["_1", "_2", "_3"]).agg({"_4":"sum"}).sort(desc("sum(_4)")).toPandas()
        myrows["_2"] = myrows._2.astype("float")
        myrows["_3"] = myrows._3.astype("float")
        print(myrows.head(5))
        #plot sentiment of top words by count
        sn.plt.figure( figsize = ( 20, 20 ) )
        sn.lmplot("_2", 
               "_3", 
               data=myrows.head(50), 
               fit_reg=False, 
               hue = "_2",
               palette = "husl", 
               legend = False, 
               size = 5,
               scatter = True, 
               scatter_kws={"marker": "D", 
                            "s": myrows["sum(_4)"] * 30}) 
        plt.title("Arousal vs. Dominance")
        plt.xlabel("Arousal Score")
        plt.ylabel("Dominance Score")
        sn.plt.ylim(0,)
        sn.plt.xlim(0,)
        sn.plt.show()
        count = count + 1
    except ValueError as e:
        print(e) 

2018-12-06 00:29:39.925885


AnalysisException: u'Table or view not found: tweets; line 1 pos 14'

In [ ]:
ssc.stop()